In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [2]:
# Check for GPU
import tensorflow as tf
try:
    from google.colab import drive
    IN_COLAB=True
except:
    IN_COLAB=False

if IN_COLAB:
    print("We're running Colab")
else:
    print(tf.config.list_physical_devices())
    print('\nCUDA GPU: ' + str(tf.test.is_gpu_available(cuda_only=True)))

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.

CUDA GPU: True


# Data Preprocessing

In [3]:
import os
os.chdir('..')

df = pd.read_csv('./hourly02-ithaca/hourly02-NY_Ithaca_13_E.csv', header = 0, index_col = 0)

In [4]:
df

,WBANNO,UTC_DATE,UTC_TIME,LST_DATE,LST_TIME,CRX_VN,LONGITUDE,LATITUDE,T_CALC,T_HR_AVG,...,SOIL_MOISTURE_5,SOIL_MOISTURE_10,SOIL_MOISTURE_20,SOIL_MOISTURE_50,SOIL_MOISTURE_100,SOIL_TEMP_5,SOIL_TEMP_10,SOIL_TEMP_20,SOIL_TEMP_50,SOIL_TEMP_100
0,64758,20041027,2200,20041027,1700,1.201,-76.25,42.44,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,64758,20041027,2300,20041027,1800,1.201,-76.25,42.44,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,64758,20041028,0,20041027,1900,1.201,-76.25,42.44,7.8,7.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,64758,20041028,100,20041027,2000,1.201,-76.25,42.44,6.5,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,64758,20041028,200,20041027,2100,1.201,-76.25,42.44,5.4,6.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7415,64758,20231106,0,20231105,1900,2.622,-76.25,42.44,1.1,1.3,...,0.336,0.308,0.307,0.309,0.023,7.6,8.2,8.8,9.1,10.4
7416,64758,20231106,100,20231105,2000,2.622,-76.25,42.44,-0.1,0.0,...,0.336,0.309,0.307,0.309,0.021,7.3,8.0,8.8,9.1,10.4
7417,64758,20231106,200,20231105,2100,2.622,-76.25,42.44,-0.5,-0.4,...,0.336,0.309,0.308,0.309,0.022,7.0,7.9,8.7,9.2,10.4
7418,64758,20231106,300,20231105,2200,2.622,-76.25,42.44,-1.4,-1.3,...,0.336,0.309,0.308,0.309,0.022,6.8,7.7,8.6,9.1,10.6


In [5]:
Date = pd.to_datetime(df.UTC_DATE, format='%Y%m%d', errors='coerce')
+ pd.to_timedelta(df.UTC_TIME//100, unit = 'hours')
df['Time'] = Date

In [6]:
df.columns

Index(['WBANNO', 'UTC_DATE', 'UTC_TIME', 'LST_DATE', 'LST_TIME', 'CRX_VN',
       'LONGITUDE', 'LATITUDE', 'T_CALC', 'T_HR_AVG', 'T_MAX', 'T_MIN',
       'P_CALC', 'SOLARAD', 'SOLARAD_FLAG', 'SOLARAD_MAX', 'SOLARAD_MAX_FLAG',
       'SOLARAD_MIN', 'SOLARAD_MIN_FLAG', 'SUR_TEMP_TYPE', 'SUR_TEMP',
       'SUR_TEMP_FLAG', 'SUR_TEMP_MAX', 'SUR_TEMP_MAX_FLAG', 'SUR_TEMP_MIN',
       'SUR_TEMP_MIN_FLAG', 'RH_HR_AVG', 'RH_HR_AVG_FLAG', 'SOIL_MOISTURE_5',
       'SOIL_MOISTURE_10', 'SOIL_MOISTURE_20', 'SOIL_MOISTURE_50',
       'SOIL_MOISTURE_100', 'SOIL_TEMP_5', 'SOIL_TEMP_10', 'SOIL_TEMP_20',
       'SOIL_TEMP_50', 'SOIL_TEMP_100', 'Time'],
      dtype='object')

In [7]:
data = df[['T_CALC', 'T_HR_AVG', 'T_MAX', 'T_MIN',
       'P_CALC', 'SOLARAD', 'SOLARAD_MAX',
       'SOLARAD_MIN', 'SUR_TEMP',
           'SUR_TEMP_MAX', 'SUR_TEMP_MIN', 'RH_HR_AVG']]

In [8]:
data.index = df['Time']

In [9]:
# check for N/A
data.min()

T_CALC         -30.8
T_HR_AVG       -29.3
T_MAX          -28.4
T_MIN          -30.9
P_CALC           0.0
SOLARAD          0.0
SOLARAD_MAX      0.0
SOLARAD_MIN      0.0
SUR_TEMP       -35.8
SUR_TEMP_MAX   -61.0
SUR_TEMP_MIN   -36.0
RH_HR_AVG        0.0
dtype: float64

In [10]:
data

,T_CALC,T_HR_AVG,T_MAX,T_MIN,P_CALC,SOLARAD,SOLARAD_MAX,SOLARAD_MIN,SUR_TEMP,SUR_TEMP_MAX,SUR_TEMP_MIN,RH_HR_AVG
Time,,,,,,,,,,,,
2004-10-27,NaN,NaN,NaN,NaN,NaN,27.0,NaN,NaN,8.8,NaN,NaN,0.0
2004-10-27,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,6.7,NaN,NaN,0.0
2004-10-28,7.8,7.6,8.0,7.3,0.0,0.0,NaN,NaN,6.1,NaN,NaN,0.0
2004-10-28,6.5,7.0,7.8,6.5,0.0,0.0,NaN,NaN,5.6,NaN,NaN,0.0
2004-10-28,5.4,6.2,6.5,5.4,0.0,0.0,NaN,NaN,5.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-06,1.1,1.3,2.1,0.3,0.0,0.0,0.0,0.0,-1.6,-0.9,-2.0,76.0
2023-11-06,-0.1,0.0,1.0,-1.1,0.0,0.0,0.0,0.0,-2.3,-1.9,-2.8,80.0
2023-11-06,-0.5,-0.4,0.1,-0.7,0.0,0.0,0.0,0.0,-3.1,-2.8,-3.3,83.0


In [11]:
data.isna().sum()

T_CALC           1384
T_HR_AVG         1446
T_MAX            1385
T_MIN            1389
P_CALC            832
SOLARAD           570
SOLARAD_MAX      9757
SOLARAD_MIN      9757
SUR_TEMP          724
SUR_TEMP_MAX     9911
SUR_TEMP_MIN     9911
RH_HR_AVG       48248
dtype: int64

In [12]:
# Check data types
data.dtypes

T_CALC          float64
T_HR_AVG        float64
T_MAX           float64
T_MIN           float64
P_CALC          float64
SOLARAD         float64
SOLARAD_MAX     float64
SOLARAD_MIN     float64
SUR_TEMP        float64
SUR_TEMP_MAX    float64
SUR_TEMP_MIN    float64
RH_HR_AVG       float64
dtype: object

In [13]:
data.shape

(166759, 12)

In [14]:
# forward fill the missing values  
data.ffill(axis = 0, inplace = True) 

In [15]:
data

,T_CALC,T_HR_AVG,T_MAX,T_MIN,P_CALC,SOLARAD,SOLARAD_MAX,SOLARAD_MIN,SUR_TEMP,SUR_TEMP_MAX,SUR_TEMP_MIN,RH_HR_AVG
Time,,,,,,,,,,,,
2004-10-27,NaN,NaN,NaN,NaN,NaN,27.0,NaN,NaN,8.8,NaN,NaN,0.0
2004-10-27,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,6.7,NaN,NaN,0.0
2004-10-28,7.8,7.6,8.0,7.3,0.0,0.0,NaN,NaN,6.1,NaN,NaN,0.0
2004-10-28,6.5,7.0,7.8,6.5,0.0,0.0,NaN,NaN,5.6,NaN,NaN,0.0
2004-10-28,5.4,6.2,6.5,5.4,0.0,0.0,NaN,NaN,5.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-06,1.1,1.3,2.1,0.3,0.0,0.0,0.0,0.0,-1.6,-0.9,-2.0,76.0
2023-11-06,-0.1,0.0,1.0,-1.1,0.0,0.0,0.0,0.0,-2.3,-1.9,-2.8,80.0
2023-11-06,-0.5,-0.4,0.1,-0.7,0.0,0.0,0.0,0.0,-3.1,-2.8,-3.3,83.0


In [16]:
# drop NaN at the top
data.dropna(inplace = True)

In [17]:
# set target
data['target'] = data['T_HR_AVG']

In [18]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, shuffle = False)

In [19]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

In [20]:
train.shape

(126057, 13)

In [21]:
test.shape

(31515, 13)

In [22]:
# splitting data into sequences
def split_sequences(features, target, seq_len, forecast_len):
    X,y = list(), list()
    for i in range(len(features)):
        end_input = i + seq_len
        end_predict = end_input + forecast_len
        if end_predict > len(features)-1:
            break
        seq_x, seq_y = features[i:end_input,:], target[end_input:end_predict]
        X.append(seq_x)
        y.append(seq_y)
    return tf.convert_to_tensor(X, dtype=tf.float64), tf.convert_to_tensor(y, dtype=tf.float64)

# Define Model

In [23]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, RNN, LSTMCell, Input, Bidirectional
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import plot_model

class MyModel(tf.keras.Model):

    def __init__(self, input_shape, output_shape, name = 'LSTM-FC'):
        super().__init__(name = name)
        self.input_layer = Input(shape = input_shape, name = 'input')
        self.lstm1 = LSTM(units=30, activation = 'tanh', input_shape = input_shape, return_sequences=False, name = 'lstm_1')
        self.dense1 = Dense(units=20, activation = 'relu', name = 'dense_1')
        self.dense2 = Dense(units=10, activation = 'relu', name = 'dense_2')
        self.dense3 = Dense(units = output_shape, activation = 'sigmoid', name = 'dense_3')
        #self.dropout = tf.keras.layers.Dropout(0.5)

    def call(self, inputs, training=False):
        x = self.lstm1(inputs)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        #if training:
        #  x = self.dropout(x, training=training)
        return x
    
    def summary(self):
        model = Model(inputs = [self.input_layer], outputs = self.call(self.input_layer), name = self.name)
        return model.summary()

# Model Training
## input length : output length = 16:4

In [24]:
# prepare sequences
seq_len = 16
forecast_len = 4
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [25]:
X_train.shape

TensorShape([126037, 16, 12])

In [26]:
y_train.shape

TensorShape([126037, 4])

In [27]:
# create model instance
model_name = 'LSTM-FC_16-4'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "LSTM-FC_16-4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 16, 12)]          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 30)                5160      
_________________________________________________________________
dense_1 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 44        
Total params: 6,034
Trainable params: 6,034
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train, 
          y_train, 
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

# save trained model
model.save('./LSTM/models/' + model_name)

Epoch 1/30
1135/1135 [==============================] - 13s 9ms/step - loss: 0.0012 - mse: 0.0012 - acc: 0.5799 - val_loss: 6.2373e-04 - val_mse: 6.2373e-04 - val_acc: 0.6089
Epoch 2/30
1135/1135 [==============================] - 9s 8ms/step - loss: 6.2709e-04 - mse: 6.2709e-04 - acc: 0.6399 - val_loss: 5.9543e-04 - val_mse: 5.9543e-04 - val_acc: 0.6054
Epoch 3/30
1135/1135 [==============================] - 10s 8ms/step - loss: 5.7441e-04 - mse: 5.7441e-04 - acc: 0.6491 - val_loss: 7.6644e-04 - val_mse: 7.6644e-04 - val_acc: 0.6133
Epoch 4/30
1135/1135 [==============================] - 10s 9ms/step - loss: 5.4899e-04 - mse: 5.4899e-04 - acc: 0.6562 - val_loss: 4.7159e-04 - val_mse: 4.7159e-04 - val_acc: 0.5918
Epoch 5/30
1135/1135 [==============================] - 10s 8ms/step - loss: 5.2823e-04 - mse: 5.2823e-04 - acc: 0.6503 - val_loss: 4.8448e-04 - val_mse: 4.8448e-04 - val_acc: 0.6109
Epoch 6/30
1135/1135 [==============================] - 10s 9ms/step - loss: 5.1569e-04 - mse:

INFO:tensorflow:Assets written to: ./LSTM/models/LSTM-FC_16-4\assets


INFO:tensorflow:Assets written to: ./LSTM/models/LSTM-FC_16-4\assets


In [29]:
y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

In [30]:
from sklearn.metrics import mean_squared_error
print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

mean_squared_error
train set: 0.0004253659562950462
test set: 0.00046580697737957217


## input length : output length = 24:6

In [31]:
# reset memory
tf.Graph().as_default() 

# prepare sequences
seq_len = 24
forecast_len = 6
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [32]:
# create model instance
model_name = 'LSTM-FC_24-6'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "LSTM-FC_24-6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 24, 12)]          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 30)                5160      
_________________________________________________________________
dense_1 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 66        
Total params: 6,056
Trainable params: 6,056
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train, 
          y_train, 
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

# save trained model
model.save('./LSTM/models/' + model_name)

Epoch 1/30
1135/1135 [==============================] - 13s 10ms/step - loss: 0.0018 - mse: 0.0018 - acc: 0.4746 - val_loss: 9.0575e-04 - val_mse: 9.0575e-04 - val_acc: 0.5967
Epoch 2/30
1135/1135 [==============================] - 11s 10ms/step - loss: 8.4904e-04 - mse: 8.4904e-04 - acc: 0.5992 - val_loss: 7.6280e-04 - val_mse: 7.6280e-04 - val_acc: 0.5841
Epoch 3/30
1135/1135 [==============================] - 10s 9ms/step - loss: 7.8417e-04 - mse: 7.8417e-04 - acc: 0.6001 - val_loss: 7.9128e-04 - val_mse: 7.9128e-04 - val_acc: 0.5978
Epoch 4/30
1135/1135 [==============================] - 10s 9ms/step - loss: 7.5774e-04 - mse: 7.5774e-04 - acc: 0.6022 - val_loss: 7.2393e-04 - val_mse: 7.2393e-04 - val_acc: 0.5515
Epoch 5/30
1135/1135 [==============================] - 10s 9ms/step - loss: 7.2353e-04 - mse: 7.2353e-04 - acc: 0.6057 - val_loss: 7.2187e-04 - val_mse: 7.2187e-04 - val_acc: 0.6018
Epoch 6/30
1135/1135 [==============================] - 10s 9ms/step - loss: 7.1014e-04 - m

INFO:tensorflow:Assets written to: ./LSTM/models/LSTM-FC_24-6\assets


INFO:tensorflow:Assets written to: ./LSTM/models/LSTM-FC_24-6\assets


In [34]:
# evaluate the model
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

mean_squared_error
train set: 0.0005667976831485296
test set: 0.0006435656014014288


## input length : output length = 32:8

In [35]:
# reset memory
tf.Graph().as_default() 

# prepare sequences
seq_len = 32
forecast_len = 8
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [36]:
# create model instance
model_name = 'LSTM-FC_32-8'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "LSTM-FC_32-8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 32, 12)]          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 30)                5160      
_________________________________________________________________
dense_1 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 88        
Total params: 6,078
Trainable params: 6,078
Non-trainable params: 0
_________________________________________________________________


In [37]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train, 
          y_train, 
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

# save trained model
model.save('./LSTM/models/' + model_name)

Epoch 1/30
1135/1135 [==============================] - 10s 8ms/step - loss: 0.0020 - mse: 0.0020 - acc: 0.4857 - val_loss: 0.0018 - val_mse: 0.0018 - val_acc: 0.5547
Epoch 2/30
1135/1135 [==============================] - 8s 7ms/step - loss: 0.0012 - mse: 0.0012 - acc: 0.5531 - val_loss: 0.0011 - val_mse: 0.0011 - val_acc: 0.5603
Epoch 3/30
1135/1135 [==============================] - 8s 7ms/step - loss: 0.0011 - mse: 0.0011 - acc: 0.5558 - val_loss: 0.0011 - val_mse: 0.0011 - val_acc: 0.5382
Epoch 4/30
1135/1135 [==============================] - 8s 7ms/step - loss: 0.0011 - mse: 0.0011 - acc: 0.5581 - val_loss: 0.0017 - val_mse: 0.0017 - val_acc: 0.5213
Epoch 5/30
1135/1135 [==============================] - 8s 7ms/step - loss: 0.0010 - mse: 0.0010 - acc: 0.5597 - val_loss: 0.0010 - val_mse: 0.0010 - val_acc: 0.5363
Epoch 6/30
1135/1135 [==============================] - 8s 7ms/step - loss: 0.0010 - mse: 0.0010 - acc: 0.5609 - val_loss: 0.0011 - val_mse: 0.0011 - val_acc: 0.5597
Epo

INFO:tensorflow:Assets written to: ./LSTM/models/LSTM-FC_32-8\assets


INFO:tensorflow:Assets written to: ./LSTM/models/LSTM-FC_32-8\assets


In [38]:
# evaluate the model
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

mean_squared_error
train set: 0.0008076461036605471
test set: 0.0009041629087394275


## input length : output length = 40:10

In [39]:
# reset memory
tf.Graph().as_default() 

# prepare sequences
seq_len = 40
forecast_len = 10
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [40]:
# create model instance
model_name = 'LSTM-FC_40-10'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "LSTM-FC_40-10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 40, 12)]          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 30)                5160      
_________________________________________________________________
dense_1 (Dense)              (None, 20)                620       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                110       
Total params: 6,100
Trainable params: 6,100
Non-trainable params: 0
_________________________________________________________________


In [41]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train, 
          y_train, 
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

# save trained model
model.save('./LSTM/models/' + model_name)

Epoch 1/30
1135/1135 [==============================] - 10s 8ms/step - loss: 0.0024 - mse: 0.0024 - acc: 0.4314 - val_loss: 0.0020 - val_mse: 0.0020 - val_acc: 0.4466
Epoch 2/30
1135/1135 [==============================] - 8s 7ms/step - loss: 0.0015 - mse: 0.0015 - acc: 0.4946 - val_loss: 0.0020 - val_mse: 0.0020 - val_acc: 0.4327
Epoch 3/30
1135/1135 [==============================] - 8s 7ms/step - loss: 0.0014 - mse: 0.0014 - acc: 0.5039 - val_loss: 0.0014 - val_mse: 0.0014 - val_acc: 0.5074
Epoch 4/30
1135/1135 [==============================] - 8s 7ms/step - loss: 0.0013 - mse: 0.0013 - acc: 0.5074 - val_loss: 0.0014 - val_mse: 0.0014 - val_acc: 0.4696
Epoch 5/30
1135/1135 [==============================] - 8s 7ms/step - loss: 0.0013 - mse: 0.0013 - acc: 0.5104 - val_loss: 0.0014 - val_mse: 0.0014 - val_acc: 0.5068
Epoch 6/30
1135/1135 [==============================] - 8s 7ms/step - loss: 0.0012 - mse: 0.0012 - acc: 0.5137 - val_loss: 0.0013 - val_mse: 0.0013 - val_acc: 0.5058
Epo

INFO:tensorflow:Assets written to: ./LSTM/models/LSTM-FC_40-10\assets


INFO:tensorflow:Assets written to: ./LSTM/models/LSTM-FC_40-10\assets


In [42]:
# evaluate the model
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

mean_squared_error
train set: 0.001081192817213594
test set: 0.0012372215753089972
